In [5]:
import pandas as pd
from pyairtable import Api

In [23]:
rfm = pd.read_json('https://extrem.ishukshin.ru/json')
rfm = expected.convert_dtypes()
rfm.is_ukrainian = expected.is_ukrainian.astype('boolean')
rfm.head()

,id,name_full,names,is_terr,birth,place,is_ukrainian
0,1,АБАДИЕВ МАГОМЕД МИКАИЛОВИЧ*,[АБАДИЕВ МАГОМЕД МИКАИЛОВИЧ],True,09.11.1982 г.р.,Г. НАЗРАНЬ ЧИАССР,False
1,3,АБАЕВ ИДРИС МОВСАРОВИЧ*,[АБАЕВ ИДРИС МОВСАРОВИЧ],True,02.01.1986 г.р.,С. СУВОРОВ-ЮРТ ГУДЕРМЕССКОГО РАЙОНА ЧИАССР,False
2,6,АБАЗОВ АХМЕД ЭДУАРДОВИЧ*,[АБАЗОВ АХМЕД ЭДУАРДОВИЧ],True,30.08.1992 г.р.,С. КЫЗБУРУН-3 БАКСАНСКОГО РАЙОНА КАБАРДИНО-БАЛ...,False
3,7,АБАЗОВ БЕСЛАН СУПЬЯНОВИЧ*,[АБАЗОВ БЕСЛАН СУПЬЯНОВИЧ],True,15.08.1993 г.р.,С. ШАЛИ ШАЛИНСКОГО РАЙОНА ЧЕЧЕНСКОЙ РЕСПУБЛИКИ,False
4,8,АБАЗОВ МУРАТ АРСЕНОВИЧ*,[АБАЗОВ МУРАТ АРСЕНОВИЧ],True,10.05.1989 г.р.,Г. НАЛЬЧИК КАБАРДИНО-БАЛКАРСКОЙ РЕСПУБЛИКИ,False


In [11]:
api = Api('patoCRTkfxdzWWT0Y.f20c7fccca48bbbbffe5d1d993f8415ebc5fd3b2d95b4a15ab38e590dc949202')
pzk = pd.json_normalize(api.table('app1sDXWAmADet7vo', 'tbl321Qu3l3aQfvYp').all())
pzk.rename(columns={
    'id': 'id',
    'createdTime': 'created_time',
    'fields.++ФИО': 'name',
    'fields.Дело': 'case',
    'fields.Регион' : 'region',
    'fields.++Город' : 'town',
    'fields.Возраст' : 'age',
    'fields.Росфинмониторинг' : 'rfm',
    'fields.Дата включения в список РФМ' : 'date_rfm_added',
    'fields.Дата исключения из списка РФМ' : 'date_rfm_removed'
}, inplace=True)
pzk.drop(columns=['id', 'created_time'], inplace=True)
pzk = pzk.convert_dtypes()
pzk.date_rfm_added = pd.to_datetime(pzk.date_rfm_added)
pzk.date_rfm_removed = pd.to_datetime(pzk.date_rfm_removed)
pzk.head()

,case,name,town,age,region,date_rfm_added,rfm,date_rfm_removed
0,Дело о Маяковских чтениях,Арифджанов Назар Рустемович,Москва,25,Москва,NaT,<NA>,NaT
1,Дело против беженца Ильшата Шарафуллина,Шарафуллин Ильшат Маратович,Набережные Челны,48,Республика Татарстан,2023-01-31,True,NaT
2,Дворцовое дело,Коваленко Ренат Газизович,Новосибирск,30,Новосибирская область,2021-03-19,True,2023-10-12
3,Дело о подготовке теракта на День победы во вр...,Запорожец Павел Павлович,Скадовск,33,Херсонская область,2022-08-19,True,NaT
4,Обыски у активистов Объединения перевозчиков Р...,Понеж Сергей,Санкт-Петербург,<NA>,Санкт-Петербург,NaT,<NA>,NaT


In [26]:
pzk.merge(rfm, left_on=pzk.name.str.lower(), right_on=rfm.name_full.str.lower().str.rstrip('*'))

,key_0,case,name,town,age,region,date_rfm_added,rfm,date_rfm_removed,id,name_full,names,is_terr,birth,place,is_ukrainian
0,шарафуллин ильшат маратович,Дело против беженца Ильшата Шарафуллина,Шарафуллин Ильшат Маратович,Набережные Челны,48,Республика Татарстан,2023-01-31,True,NaT,16971,ШАРАФУЛЛИН ИЛЬШАТ МАРАТОВИЧ*,[ШАРАФУЛЛИН ИЛЬШАТ МАРАТОВИЧ],True,10.05.1975 г.р.,Г. НАБЕРЕЖНЫЕ ЧЕЛНЫ РЕСПУБЛИКИ ТАТАРСТАН,False
1,запорожец павел павлович,Дело о подготовке теракта на День победы во вр...,Запорожец Павел Павлович,Скадовск,33,Херсонская область,2022-08-19,True,NaT,16026,ЗАПОРОЖЕЦ ПАВЕЛ ПАВЛОВИЧ*,[ЗАПОРОЖЕЦ ПАВЕЛ ПАВЛОВИЧ],True,19.05.1990 г.р.,Г. СКАДОВСК ХЕРСОНСКОЙ ОБЛАСТИ УКРАИНЫ,True
2,лепишин александр александрович,Дело троих членов «Хизб ут-Тахрир» в Челябинске,Лепишин Александр Александрович,Челябинск,<NA>,Челябинская область,NaT,<NA>,NaT,4338,ЛЕПИШИН АЛЕКСАНДР АЛЕКСАНДРОВИЧ*,[ЛЕПИШИН АЛЕКСАНДР АЛЕКСАНДРОВИЧ],True,06.10.1988 г.р.,Г. ЧЕЛЯБИНСК ЧЕЛЯБИНСКОЙ ОБЛАСТИ,False
3,хабиров вадим байегитович,Челябинское дело 4 о членстве в запрещённой «Х...,Хабиров Вадим Байегитович,Челябинск,45,Челябинская область,NaT,<NA>,NaT,9517,ХАБИРОВ ВАДИМ БАЙЕГИТОВИЧ*,[ХАБИРОВ ВАДИМ БАЙЕГИТОВИЧ],True,01.07.1978 г.р.,ПОС. ЧИСТЫЙ КРАСНОАРМЕЙСКОГО РАЙОНА ЧЕЛЯБИНСКО...,False
4,темерьянов вилен фаридович,Первая джанкойская группа запрещенной «Хизб ут...,Темерьянов Вилен Фаридович,Джанкой,39,Республика Крым,NaT,<NA>,NaT,16079,ТЕМЕРЬЯНОВ ВИЛЕН ФАРИДОВИЧ*,[ТЕМЕРЬЯНОВ ВИЛЕН ФАРИДОВИЧ],True,16.01.1985 г.р.,Г. КАПСАН КАШКАДАРЬИНСКОЙ ОБЛАСТИ УЗБЕКСКОЙ ССР,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,лаврентьев леонид викторович,Дело «Комитет безопасности России (КОБРа)»,Лаврентьев Леонид Викторович,Волжский,62,Волгоградская область,2023-05-30,True,NaT,17715,ЛАВРЕНТЬЕВ ЛЕОНИД ВИКТОРОВИЧ,[ЛАВРЕНТЬЕВ ЛЕОНИД ВИКТОРОВИЧ],False,25.12.1961 г.р.,ГОР. ВОЛЖСКИЙ ВОЛГОГРАДСКОЙ ОБЛАСТИ,False
1643,алсынбаев шафкат салимьянович,Сибайское дело троих о членстве в запрещённой ...,Алсынбаев Шафкат Салимьянович,<NA>,33,Республика Башкортостан,2018-08-08,True,NaT,9354,АЛСЫНБАЕВ ШАФКАТ САЛИМЬЯНОВИЧ*,[АЛСЫНБАЕВ ШАФКАТ САЛИМЬЯНОВИЧ],True,26.10.1990 г.р.,С. ЯКОВЛЕВКА БАЙМАКСКОГО РАЙОНА РЕСПУБЛИКИ БАШ...,False
1644,ивашин игорь николаевич,Дело Ивашина в Ленске,Ивашин Игорь Николаевич,<NA>,47,<NA>,NaT,<NA>,NaT,11585,ИВАШИН ИГОРЬ НИКОЛАЕВИЧ,[ИВАШИН ИГОРЬ НИКОЛАЕВИЧ],False,16.04.1976 г.р.,Г. ЛЕНСК ЛЕНСКОГО РАЙОНА ЯКУТСКОЙ АССР,False
1645,месутов руслан аметович,Алуштинское дело о членстве в запрещённой «Хиз...,Месутов Руслан Аметович,Алушта,58,Республика Крым,NaT,<NA>,NaT,11042,МЕСУТОВ РУСЛАН АМЕТОВИЧ*,[МЕСУТОВ РУСЛАН АМЕТОВИЧ],True,11.09.1965 г.р.,Г. КАТГАКУРГАП САМАРКАНДСКОЙ ОБЛАСТИ УЗБЕКСКОЙ...,False
